In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data
from sklearn import svm
import time
from elm import GenELMClassifier
from random_layer import MLPRandomLayer


/home/kabir/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
NUMBER_OF_FEATURES = 128
BATCH_SIZE = 55
BATCHES_IN_EPOCH = 100
TRAIN_SIZE = 5500
TEST_SIZE = 2000
NUMBER_OF_EPOCHS = 3
NUMBER_OF_EXPERIMENTS = 1

In [3]:
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
converter = np.array([0,1,2,3,4,5,6,7,8,9])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
experiment_results = {
    "ConvNet-ACCU":0,}

In [5]:

train_features = np.zeros((TRAIN_SIZE, 28*28), dtype=float)
train_labels = np.zeros(TRAIN_SIZE, dtype=int)
test_features = mnist.test.images
print(train_features.shape)
print(test_features.shape)
test_labels = np.zeros(TEST_SIZE, dtype=int)


(5500, 784)
(10000, 784)


In [6]:
train_features_cnn = np.zeros((TRAIN_SIZE, NUMBER_OF_FEATURES), dtype=float)
train_labels_cnn = np.zeros(TRAIN_SIZE, dtype=int)
test_labels_cnn = np.zeros(TEST_SIZE, dtype=int)

In [7]:
def print_debug(ndarrayinput, stringinput):
    print("\n"+stringinput)
    print(ndarrayinput.shape)
    print(type(ndarrayinput))
    print(np.mean(ndarrayinput))
    print(ndarrayinput)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



In [16]:

def ConvNet(number_of_training_epochs):
    print("\n#########################\nConvNet Train/Test\n#########################")
    initial_time = time.time()

    for i in range(number_of_training_epochs * BATCHES_IN_EPOCH):
        batch = mnist.train.next_batch(BATCH_SIZE)
        if i%BATCHES_IN_EPOCH == 0:
            train_accuracy = model_accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
#            print("\nEpoch ", int(i/BATCHES_IN_EPOCH), "Training Accuracy ", train_accuracy)
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    training_time = time.time()-initial_time
    print("\nTraining Time = ", training_time)

    accuracy = model_accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
#    test_time = time.time() - (training_time + initial_time)
#    print("\nTest Time = ", test_time)

    print("\nConvNet accuracy =", accuracy)
    return accuracy, training_time

In [15]:
print("\n#########################\nStarting\n#########################\n")

# sess = tf.InteractiveSession()

print("\n#########################\nBuilding ConvNet\n#########################")

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])


sess.run(tf.initialize_all_variables())
x_image = tf.reshape(x, [-1,28,28,1])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
W_fc1 = weight_variable([7 * 7 * 64, NUMBER_OF_FEATURES])
b_fc1 = bias_variable([NUMBER_OF_FEATURES])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([NUMBER_OF_FEATURES, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
model_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





#########################
Starting
#########################


#########################
Building ConvNet
#########################


In [17]:
print("\n#########################\nExecuting Experiments\n#########################")

dataframe_results = pd.DataFrame()

for index in range(NUMBER_OF_EXPERIMENTS):
    print("\n#########################\nExperiment", index+1, "of", NUMBER_OF_EXPERIMENTS, "\n#########################")
    sess.run(tf.initialize_all_variables())
    experiment_results["ConvNet-ACCU"], experiment_results["ConvNet-TIME"] = ConvNet(NUMBER_OF_EPOCHS)
    dataframe_results = dataframe_results.append(experiment_results, ignore_index=True)

dataframe_results = dataframe_results[["ConvNet-ACCU","ConvNet-TIME",]]

print("\n#########################\nPrinting Results\n#########################\n")

print("\n", dataframe_results, "\n")
print(dataframe_results.describe())

print("\n#########################\nStoping\n#########################\n")

sess.close()


#########################
Executing Experiments
#########################

#########################
Experiment 1 of 1 
#########################

#########################
ConvNet Train/Test
#########################

Training Time =  37.45641851425171

ConvNet accuracy = 0.8906

#########################
Printing Results
#########################


    ConvNet-ACCU  ConvNet-TIME
0        0.8906     37.456419 

       ConvNet-ACCU  ConvNet-TIME
count        1.0000      1.000000
mean         0.8906     37.456419
std             NaN           NaN
min          0.8906     37.456419
25%          0.8906     37.456419
50%          0.8906     37.456419
75%          0.8906     37.456419
max          0.8906     37.456419

#########################
Stoping
#########################

